In [1]:
#import libraries

import pandas as pd

import numpy as np

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import catboost as cb
from catboost import CatBoostRegressor

In [2]:
# Data for training
diamonds = pd.read_csv('./diamonds.csv')

# Data for kaggle
diamonds_test = pd.read_csv('./diamonds_test.csv')

In [3]:
#Drop id and unnamed

def column_drop(x,y):
    x.drop(['Unnamed: 0'], inplace = True, axis = 1)
    x.drop(['index_id'], inplace = True, axis = 1)
    y.drop(['id'], inplace = True, axis = 1)
    
column_drop(diamonds,diamonds_test)

In [4]:
#Function to drop registers with 0 in x, y, z

def drop_0s(x):
    x = x[x['x']!=0]
    x = x[x['y']!=0]
    x = x[x['z']!=0]
    return x

In [5]:
# def proporciones correctas

def uniform(x,y):
    x['uniform'] = x['z']/ x['y']
    y['uniform'] = y['z']/ y['y']

In [6]:
# Function to avoid NaN in uniform column

def fill_na(x,y):
    x['uniform'].fillna(x['depth']/100, inplace = True)
    y['uniform'].fillna(y['depth']/100, inplace = True)

In [7]:
# definition of values to explain better colors

conditions = [
    (diamonds['color'] == 'J'),
    (diamonds['color'] == 'D'),
    (diamonds['color'] == 'E'),
    (diamonds['color'] == 'F'),
    (diamonds['color'] == 'H'),
    (diamonds['color'] == 'I'),
    (diamonds['color'] == 'G')
    ]

values = [1,1,2,2,2,2,0]

In [8]:
# def proporciones correctas testing

conditions_test = [
    (diamonds_test['color'] == 'J'),
    (diamonds_test['color'] == 'D'),
    (diamonds_test['color'] == 'E'),
    (diamonds_test['color'] == 'F'),
    (diamonds_test['color'] == 'H'),
    (diamonds_test['color'] == 'I'),
    (diamonds_test['color'] == 'G')
    ]

values_test = [1,1,2,2,2,2,0]

In [9]:
# Function to select what features we want in the model

def column_sel(x):
    y = x.columns
    for i in y:
        drop_yn = input(f'drop o no drop {i}:')
        if drop_yn == 'y':
            x.drop([i], inplace = True, axis = 1)

In [10]:
# Function to encode categorical features

encode_columns = ['cut','clarity', 'color']

def hot_encoding(x):
    y = x[encode_columns]
    y = pd.get_dummies(y)
    x.drop(encode_columns, inplace = True, axis = 1)
    x = pd.merge(x, y, left_index=True, right_index=True)
    return x

In [11]:
#Creation of the good color feature

diamonds['good_color'] = np.select(conditions, values)
diamonds_test['good_color'] = np.select(conditions_test, values_test)

In [12]:
# Drop of 0s in diamonds dataset

diamonds = drop_0s(diamonds)

In [13]:
#Creation of the uniform feature

uniform(diamonds, diamonds_test)

In [14]:
# Fillna in uniform feature in both datasets

fill_na(diamonds, diamonds_test)

In [15]:
# Feature selection diamonds

column_sel(diamonds)

drop o no drop price:n
drop o no drop carat:n
drop o no drop city:y
drop o no drop depth:y
drop o no drop table:y
drop o no drop x:y
drop o no drop y:y
drop o no drop z:y
drop o no drop cut:n
drop o no drop color:n
drop o no drop clarity:n
drop o no drop good_color:n
drop o no drop uniform:n


In [16]:
diamonds

,price,carat,cut,color,clarity,good_color,uniform
0,4268,1.21,Premium,J,VS2,1,0.625920
1,4839,1.20,Premium,J,VS2,1,0.606676
2,368,0.30,Premium,J,VS2,1,0.602740
3,5053,1.20,Premium,J,VS2,1,0.629464
4,3593,1.05,Premium,J,VS2,1,0.597264
...,...,...,...,...,...,...,...
40450,2632,0.59,Very Good,F,IF,2,0.597806
40451,9972,1.01,Very Good,F,IF,2,0.628705
40452,886,0.30,Very Good,F,IF,2,0.621810
40453,3205,0.71,Fair,F,IF,2,0.590361


In [17]:
# Feature selection diamonds test

column_sel(diamonds_test)

drop o no drop carat:n
drop o no drop cut:n
drop o no drop color:n
drop o no drop clarity:n
drop o no drop depth:y
drop o no drop table:y
drop o no drop x:y
drop o no drop y:y
drop o no drop z:y
drop o no drop city:y
drop o no drop good_color:n
drop o no drop uniform:n


In [18]:
diamonds_test

,carat,cut,color,clarity,good_color,uniform
0,0.79,Very Good,F,SI1,2,0.623090
1,1.20,Ideal,J,VS1,1,0.606676
2,1.57,Premium,H,SI1,2,0.624317
3,0.90,Very Good,F,SI1,2,0.636215
4,0.50,Very Good,F,VS1,2,0.626719
...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,2,0.620301
13481,0.71,Ideal,I,VS2,2,0.621291
13482,0.70,Ideal,F,VS1,2,0.618214
13483,0.70,Very Good,F,SI2,2,0.585739


In [19]:
# Encoding categorical features

diamonds = hot_encoding(diamonds)
diamonds_test = hot_encoding(diamonds_test)

-------------------------------------------------------------------------------------------------------------------

**Model**

In [20]:
#we separate features from target

X = diamonds.drop(['price'], axis = 1)
y = diamonds['price']

In [21]:
#scaling

scaler = RobustScaler()
X = scaler.fit_transform(X)
X

array([[ 0.796875  , -1.        ,  0.52831587, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.78125   , -1.        , -0.71125351, ...,  0.        ,
         0.        ,  1.        ],
       [-0.625     , -1.        , -0.96482223, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.625     ,  0.        ,  0.26353222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.015625  ,  0.        , -1.76214267, ...,  0.        ,
         0.        ,  0.        ],
       [-0.515625  ,  0.        , -5.94274325, ...,  0.        ,
         0.        ,  0.        ]])

In [22]:
#Parameters to be tried in cross validation

parameters = {
    'iterations'    : [30, 50, 100,1000,5000]   
}

In [23]:
# Model definition

model = CatBoostRegressor()

In [24]:
#GridSearchCV to look for best parameters

cv = GridSearchCV(model,parameters,cv=5,verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)
cv.fit(X,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Learning rate set to 0.5
0:	learn: 2397.1485600	total: 78.8ms	remaining: 3.86s
1:	learn: 1575.1275772	total: 84.5ms	remaining: 2.03s
2:	learn: 1149.5389629	total: 92ms	remaining: 1.44s
3:	learn: 962.1880190	total: 98.4ms	remaining: 1.13s
4:	learn: 864.2092429	total: 103ms	remaining: 930ms
5:	learn: 804.6412491	total: 111ms	remaining: 817ms
6:	learn: 764.1972802	total: 117ms	remaining: 718ms
7:	learn: 738.0455393	total: 123ms	remaining: 647ms
8:	learn: 708.7712077	total: 128ms	remaining: 583ms
9:	learn: 696.0118448	total: 135ms	remaining: 540ms
10:	learn: 682.7216465	total: 139ms	remaining: 494ms
11:	learn: 672.9160588	total: 146ms	remaining: 463ms
12:	learn: 662.8293468	total: 150ms	remaining: 427ms
13:	learn: 657.4754841	total: 155ms	remaining: 397ms
14:	learn: 648.6074800	total: 161ms	remaining: 376ms
15:	learn: 639.2095357	total: 166ms	remaining: 352ms
16:	learn: 632.3578746	total: 172ms	remaining: 335ms
17:	learn: 627.8122

Learning rate set to 0.5
0:	learn: 2358.1136043	total: 57.1ms	remaining: 2.8s
1:	learn: 1605.1798554	total: 62.8ms	remaining: 1.51s
2:	learn: 1240.0854753	total: 67.8ms	remaining: 1.06s
3:	learn: 1068.3229410	total: 73.1ms	remaining: 841ms
4:	learn: 967.8033134	total: 80.3ms	remaining: 723ms
5:	learn: 896.3317234	total: 86.4ms	remaining: 634ms
6:	learn: 842.8556162	total: 90.7ms	remaining: 557ms
7:	learn: 795.6200475	total: 97.1ms	remaining: 510ms
8:	learn: 761.1002040	total: 102ms	remaining: 465ms
9:	learn: 741.3135689	total: 108ms	remaining: 432ms
10:	learn: 720.5039633	total: 115ms	remaining: 409ms
11:	learn: 698.5246532	total: 120ms	remaining: 379ms
12:	learn: 686.5921214	total: 126ms	remaining: 360ms
13:	learn: 675.4604917	total: 131ms	remaining: 337ms
14:	learn: 664.5383549	total: 136ms	remaining: 318ms
15:	learn: 650.1070702	total: 143ms	remaining: 303ms
16:	learn: 645.9578291	total: 148ms	remaining: 287ms
17:	learn: 638.0099036	total: 153ms	remaining: 272ms
18:	learn: 629.39327

253:	learn: 546.6480474	total: 1.63s	remaining: 4.78s
254:	learn: 546.1545533	total: 1.63s	remaining: 4.78s
255:	learn: 545.7865142	total: 1.64s	remaining: 4.76s
256:	learn: 545.6010198	total: 1.65s	remaining: 4.76s
257:	learn: 545.3030316	total: 1.65s	remaining: 4.75s
258:	learn: 544.9946888	total: 1.66s	remaining: 4.74s
259:	learn: 544.5443011	total: 1.66s	remaining: 4.73s
260:	learn: 544.3610207	total: 1.67s	remaining: 4.72s
261:	learn: 544.2359512	total: 1.67s	remaining: 4.71s
262:	learn: 543.8327194	total: 1.68s	remaining: 4.71s
263:	learn: 543.5839966	total: 1.68s	remaining: 4.69s
264:	learn: 543.2872683	total: 1.69s	remaining: 4.68s
265:	learn: 543.0889676	total: 1.69s	remaining: 4.67s
266:	learn: 542.8615975	total: 1.7s	remaining: 4.67s
267:	learn: 542.6408751	total: 1.71s	remaining: 4.66s
268:	learn: 542.4973863	total: 1.71s	remaining: 4.65s
269:	learn: 542.2315461	total: 1.72s	remaining: 4.64s
270:	learn: 542.0487915	total: 1.72s	remaining: 4.64s
271:	learn: 541.7303537	total

150:	learn: 946.6853569	total: 974ms	remaining: 31.3s
151:	learn: 943.3680739	total: 981ms	remaining: 31.3s
152:	learn: 940.0254618	total: 986ms	remaining: 31.3s
153:	learn: 936.6460707	total: 992ms	remaining: 31.2s
154:	learn: 933.2254402	total: 1s	remaining: 31.3s
155:	learn: 929.9627779	total: 1.01s	remaining: 31.2s
156:	learn: 927.0386344	total: 1.01s	remaining: 31.2s
157:	learn: 924.2512850	total: 1.02s	remaining: 31.2s
158:	learn: 921.5891788	total: 1.02s	remaining: 31.2s
159:	learn: 918.4582535	total: 1.03s	remaining: 31.1s
160:	learn: 915.8274590	total: 1.03s	remaining: 31.1s
161:	learn: 912.7247403	total: 1.04s	remaining: 31.1s
162:	learn: 909.8067734	total: 1.05s	remaining: 31.1s
163:	learn: 906.9892651	total: 1.05s	remaining: 31s
164:	learn: 904.4527293	total: 1.06s	remaining: 31.1s
165:	learn: 901.9392528	total: 1.06s	remaining: 31s
166:	learn: 899.2500771	total: 1.07s	remaining: 31s
167:	learn: 895.7594716	total: 1.08s	remaining: 31s
168:	learn: 893.3703837	total: 1.08s	re

450:	learn: 521.3387299	total: 3.33s	remaining: 4.05s
451:	learn: 521.1986865	total: 3.33s	remaining: 4.04s
452:	learn: 521.1137561	total: 3.34s	remaining: 4.04s
453:	learn: 521.0534640	total: 3.35s	remaining: 4.03s
454:	learn: 520.7429470	total: 3.36s	remaining: 4.02s
455:	learn: 520.6768266	total: 3.37s	remaining: 4.01s
456:	learn: 520.5959768	total: 3.37s	remaining: 4.01s
457:	learn: 520.5426119	total: 3.38s	remaining: 4s
458:	learn: 520.4620618	total: 3.39s	remaining: 3.99s
459:	learn: 520.3464784	total: 3.4s	remaining: 3.98s
460:	learn: 520.2737526	total: 3.4s	remaining: 3.98s
461:	learn: 520.1572615	total: 3.41s	remaining: 3.97s
462:	learn: 519.9821520	total: 3.42s	remaining: 3.96s
463:	learn: 519.8897290	total: 3.42s	remaining: 3.96s
464:	learn: 519.8428608	total: 3.43s	remaining: 3.95s
465:	learn: 519.6149134	total: 3.44s	remaining: 3.94s
466:	learn: 519.4308975	total: 3.45s	remaining: 3.93s
467:	learn: 519.3305688	total: 3.45s	remaining: 3.92s
468:	learn: 519.2868733	total: 3.

...

4667:	learn: 446.3218326	total: 26.6s	remaining: 1.89s
4668:	learn: 446.3095309	total: 26.6s	remaining: 1.88s
4669:	learn: 446.3030331	total: 26.6s	remaining: 1.88s
4670:	learn: 446.2900691	total: 26.6s	remaining: 1.87s
4671:	learn: 446.2784528	total: 26.6s	remaining: 1.87s
4672:	learn: 446.2714453	total: 26.6s	remaining: 1.86s
4673:	learn: 446.2592372	total: 26.6s	remaining: 1.86s
4674:	learn: 446.2558833	total: 26.6s	remaining: 1.85s
4675:	learn: 446.2471233	total: 26.6s	remaining: 1.84s
4676:	learn: 446.2338214	total: 26.6s	remaining: 1.84s
4677:	learn: 446.2174349	total: 26.6s	remaining: 1.83s
4678:	learn: 446.1968401	total: 26.6s	remaining: 1.83s
4679:	learn: 446.1835365	total: 26.6s	remaining: 1.82s
4680:	learn: 446.1710985	total: 26.6s	remaining: 1.82s
4681:	learn: 446.1634280	total: 26.7s	remaining: 1.81s
4682:	learn: 446.1513244	total: 26.7s	remaining: 1.8s
4683:	learn: 446.1439206	total: 26.7s	remaining: 1.8s
4684:	learn: 446.1424556	total: 26.7s	remaining: 1.79s
4685:	learn:

4651:	learn: 435.3369029	total: 26.5s	remaining: 1.98s
4652:	learn: 435.3287543	total: 26.5s	remaining: 1.98s
4653:	learn: 435.3143548	total: 26.5s	remaining: 1.97s
4654:	learn: 435.2994041	total: 26.5s	remaining: 1.97s
4655:	learn: 435.2855306	total: 26.5s	remaining: 1.96s
4656:	learn: 435.2772542	total: 26.6s	remaining: 1.96s
4657:	learn: 435.2569607	total: 26.6s	remaining: 1.95s
4658:	learn: 435.2388357	total: 26.6s	remaining: 1.94s
4659:	learn: 435.2246620	total: 26.6s	remaining: 1.94s
4660:	learn: 435.2164885	total: 26.6s	remaining: 1.93s
4661:	learn: 435.2102101	total: 26.6s	remaining: 1.93s
4662:	learn: 435.2004011	total: 26.6s	remaining: 1.92s
4663:	learn: 435.1945104	total: 26.6s	remaining: 1.92s
4664:	learn: 435.1792243	total: 26.6s	remaining: 1.91s
4665:	learn: 435.1571984	total: 26.6s	remaining: 1.9s
4666:	learn: 435.1502210	total: 26.6s	remaining: 1.9s
4667:	learn: 435.1457387	total: 26.6s	remaining: 1.89s
4668:	learn: 435.1439135	total: 26.6s	remaining: 1.89s
4669:	learn:

4649:	learn: 450.9096982	total: 26.4s	remaining: 1.99s
4650:	learn: 450.8920803	total: 26.4s	remaining: 1.98s
4651:	learn: 450.8719940	total: 26.4s	remaining: 1.98s
4652:	learn: 450.8714795	total: 26.4s	remaining: 1.97s
4653:	learn: 450.8631727	total: 26.4s	remaining: 1.97s
4654:	learn: 450.8435208	total: 26.5s	remaining: 1.96s
4655:	learn: 450.8385839	total: 26.5s	remaining: 1.96s
4656:	learn: 450.8259371	total: 26.5s	remaining: 1.95s
4657:	learn: 450.8220009	total: 26.5s	remaining: 1.94s
4658:	learn: 450.8070206	total: 26.5s	remaining: 1.94s
4659:	learn: 450.7862399	total: 26.5s	remaining: 1.93s
4660:	learn: 450.7781531	total: 26.5s	remaining: 1.93s
4661:	learn: 450.7597895	total: 26.5s	remaining: 1.92s
4662:	learn: 450.7418133	total: 26.5s	remaining: 1.91s
4663:	learn: 450.7213286	total: 26.5s	remaining: 1.91s
4664:	learn: 450.6963573	total: 26.5s	remaining: 1.9s
4665:	learn: 450.6654003	total: 26.5s	remaining: 1.9s
4666:	learn: 450.6551185	total: 26.5s	remaining: 1.89s
4667:	learn:

4656:	learn: 394.0749387	total: 23.2s	remaining: 1.71s
4657:	learn: 394.0616785	total: 23.2s	remaining: 1.7s
4658:	learn: 394.0569322	total: 23.2s	remaining: 1.7s
4659:	learn: 394.0346942	total: 23.2s	remaining: 1.69s
4660:	learn: 394.0128367	total: 23.2s	remaining: 1.69s
4661:	learn: 393.9999645	total: 23.2s	remaining: 1.68s
4662:	learn: 393.9919362	total: 23.2s	remaining: 1.68s
4663:	learn: 393.9915871	total: 23.2s	remaining: 1.67s
4664:	learn: 393.9816720	total: 23.2s	remaining: 1.67s
4665:	learn: 393.9733928	total: 23.2s	remaining: 1.66s
4666:	learn: 393.9634347	total: 23.2s	remaining: 1.66s
4667:	learn: 393.9525873	total: 23.2s	remaining: 1.65s
4668:	learn: 393.9353880	total: 23.2s	remaining: 1.65s
4669:	learn: 393.9188607	total: 23.2s	remaining: 1.64s
4670:	learn: 393.9030282	total: 23.2s	remaining: 1.64s
4671:	learn: 393.9026845	total: 23.2s	remaining: 1.63s
4672:	learn: 393.8771199	total: 23.2s	remaining: 1.63s
4673:	learn: 393.8638509	total: 23.2s	remaining: 1.62s
4674:	learn:

4627:	learn: 464.0793939	total: 22.9s	remaining: 1.84s
4628:	learn: 464.0723590	total: 22.9s	remaining: 1.83s
4629:	learn: 464.0591487	total: 22.9s	remaining: 1.83s
4630:	learn: 464.0493196	total: 22.9s	remaining: 1.82s
4631:	learn: 464.0453805	total: 22.9s	remaining: 1.82s
4632:	learn: 464.0176801	total: 22.9s	remaining: 1.81s
4633:	learn: 463.9948215	total: 22.9s	remaining: 1.81s
4634:	learn: 463.9884643	total: 22.9s	remaining: 1.8s
4635:	learn: 463.9816132	total: 22.9s	remaining: 1.8s
4636:	learn: 463.9659170	total: 22.9s	remaining: 1.79s
4637:	learn: 463.9626859	total: 22.9s	remaining: 1.78s
4638:	learn: 463.9615269	total: 22.9s	remaining: 1.78s
4639:	learn: 463.9583234	total: 22.9s	remaining: 1.77s
4640:	learn: 463.9526938	total: 22.9s	remaining: 1.77s
4641:	learn: 463.9479382	total: 22.9s	remaining: 1.76s
4642:	learn: 463.9323357	total: 22.9s	remaining: 1.76s
4643:	learn: 463.9228578	total: 22.9s	remaining: 1.75s
4644:	learn: 463.9181636	total: 22.9s	remaining: 1.75s
4645:	learn:

In [25]:
# Best parameters and score

print('Best hyperparameters: ', cv.best_params_, '\n')
print('Best score: ', -cv.best_score_, '\n')

Best hyperparameters:  {'iterations': 100} 

Best score:  1167.3832106885066 



In [26]:
#cross_val_score

#scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv=5, n_jobs=1)
#scores

In [32]:
# Division train/test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Model definition with best parameters

model = CatBoostRegressor(iterations = 1000)

In [34]:
# Model training

weights = model.fit(X_train, y_train)

Learning rate set to 0.070916
0:	learn: 3748.3119577	total: 3.12ms	remaining: 3.12s
1:	learn: 3522.6287096	total: 6.12ms	remaining: 3.06s
2:	learn: 3311.6545143	total: 8.73ms	remaining: 2.9s
3:	learn: 3123.1227306	total: 11.2ms	remaining: 2.78s
4:	learn: 2944.6932423	total: 14.1ms	remaining: 2.81s
5:	learn: 2778.8534616	total: 16.4ms	remaining: 2.72s
6:	learn: 2632.4782450	total: 18.7ms	remaining: 2.66s
7:	learn: 2493.2508110	total: 20.9ms	remaining: 2.58s
8:	learn: 2364.3472232	total: 23.1ms	remaining: 2.55s
9:	learn: 2246.5038968	total: 25.4ms	remaining: 2.52s
10:	learn: 2135.5089535	total: 27.7ms	remaining: 2.49s
11:	learn: 2038.9175571	total: 30.2ms	remaining: 2.48s
12:	learn: 1948.5718087	total: 32.3ms	remaining: 2.45s
13:	learn: 1865.7598147	total: 34.6ms	remaining: 2.43s
14:	learn: 1787.2230568	total: 36.9ms	remaining: 2.42s
15:	learn: 1712.8858083	total: 39.2ms	remaining: 2.41s
16:	learn: 1650.1654339	total: 41.3ms	remaining: 2.39s
17:	learn: 1589.8226818	total: 43.5ms	remainin

167:	learn: 572.8573051	total: 377ms	remaining: 1.87s
168:	learn: 572.4879780	total: 380ms	remaining: 1.87s
169:	learn: 572.0175413	total: 382ms	remaining: 1.87s
170:	learn: 571.3221205	total: 384ms	remaining: 1.86s
171:	learn: 570.8330996	total: 387ms	remaining: 1.86s
172:	learn: 570.2253729	total: 389ms	remaining: 1.86s
173:	learn: 569.3791787	total: 391ms	remaining: 1.86s
174:	learn: 568.9511419	total: 394ms	remaining: 1.86s
175:	learn: 568.3623970	total: 396ms	remaining: 1.85s
176:	learn: 567.9993099	total: 398ms	remaining: 1.85s
177:	learn: 567.5451836	total: 400ms	remaining: 1.85s
178:	learn: 566.9182588	total: 403ms	remaining: 1.85s
179:	learn: 566.4679934	total: 405ms	remaining: 1.84s
180:	learn: 565.9008835	total: 407ms	remaining: 1.84s
181:	learn: 565.4733502	total: 410ms	remaining: 1.84s
182:	learn: 564.6787199	total: 412ms	remaining: 1.84s
183:	learn: 563.9844514	total: 414ms	remaining: 1.84s
184:	learn: 563.7546210	total: 417ms	remaining: 1.83s
185:	learn: 563.2836371	tota

322:	learn: 523.4320551	total: 759ms	remaining: 1.59s
323:	learn: 523.2560159	total: 761ms	remaining: 1.59s
324:	learn: 522.9337552	total: 764ms	remaining: 1.59s
325:	learn: 522.7942896	total: 767ms	remaining: 1.58s
326:	learn: 522.5543597	total: 769ms	remaining: 1.58s
327:	learn: 522.4299776	total: 772ms	remaining: 1.58s
328:	learn: 522.3584947	total: 774ms	remaining: 1.58s
329:	learn: 522.1656980	total: 777ms	remaining: 1.58s
330:	learn: 521.9620214	total: 779ms	remaining: 1.57s
331:	learn: 521.8770060	total: 782ms	remaining: 1.57s
332:	learn: 521.8120147	total: 784ms	remaining: 1.57s
333:	learn: 521.7260373	total: 786ms	remaining: 1.57s
334:	learn: 521.5780477	total: 791ms	remaining: 1.57s
335:	learn: 521.5221772	total: 793ms	remaining: 1.57s
336:	learn: 521.3273586	total: 796ms	remaining: 1.56s
337:	learn: 521.2761110	total: 798ms	remaining: 1.56s
338:	learn: 521.0126166	total: 800ms	remaining: 1.56s
339:	learn: 520.9352953	total: 803ms	remaining: 1.56s
340:	learn: 520.8492846	tota

500:	learn: 498.6600846	total: 1.14s	remaining: 1.14s
501:	learn: 498.6087675	total: 1.15s	remaining: 1.14s
502:	learn: 498.4480065	total: 1.15s	remaining: 1.13s
503:	learn: 498.3090507	total: 1.15s	remaining: 1.13s
504:	learn: 498.1589323	total: 1.15s	remaining: 1.13s
505:	learn: 498.0467332	total: 1.16s	remaining: 1.13s
506:	learn: 497.7912949	total: 1.16s	remaining: 1.13s
507:	learn: 497.6632482	total: 1.16s	remaining: 1.12s
508:	learn: 497.5269162	total: 1.16s	remaining: 1.12s
509:	learn: 497.4943341	total: 1.16s	remaining: 1.12s
510:	learn: 497.4641521	total: 1.17s	remaining: 1.12s
511:	learn: 497.2820977	total: 1.17s	remaining: 1.11s
512:	learn: 497.1107733	total: 1.17s	remaining: 1.11s
513:	learn: 496.9681314	total: 1.18s	remaining: 1.11s
514:	learn: 496.7638788	total: 1.18s	remaining: 1.11s
515:	learn: 496.5937053	total: 1.18s	remaining: 1.11s
516:	learn: 496.3645185	total: 1.18s	remaining: 1.11s
517:	learn: 496.2738934	total: 1.19s	remaining: 1.1s
518:	learn: 496.1273709	total

667:	learn: 481.1139416	total: 1.52s	remaining: 758ms
668:	learn: 481.0477289	total: 1.53s	remaining: 756ms
669:	learn: 480.9422805	total: 1.53s	remaining: 754ms
670:	learn: 480.7293545	total: 1.53s	remaining: 752ms
671:	learn: 480.6068551	total: 1.53s	remaining: 749ms
672:	learn: 480.5606996	total: 1.54s	remaining: 747ms
673:	learn: 480.4716802	total: 1.54s	remaining: 745ms
674:	learn: 480.3864204	total: 1.54s	remaining: 743ms
675:	learn: 480.3065577	total: 1.54s	remaining: 741ms
676:	learn: 480.2837772	total: 1.55s	remaining: 739ms
677:	learn: 480.2233381	total: 1.55s	remaining: 737ms
678:	learn: 480.0769822	total: 1.55s	remaining: 734ms
679:	learn: 480.0454640	total: 1.56s	remaining: 733ms
680:	learn: 479.9712504	total: 1.56s	remaining: 731ms
681:	learn: 479.9574186	total: 1.56s	remaining: 728ms
682:	learn: 479.9404295	total: 1.56s	remaining: 726ms
683:	learn: 479.8617365	total: 1.57s	remaining: 724ms
684:	learn: 479.8376815	total: 1.57s	remaining: 722ms
685:	learn: 479.8024700	tota

840:	learn: 466.2286221	total: 1.91s	remaining: 361ms
841:	learn: 466.1506087	total: 1.91s	remaining: 359ms
842:	learn: 466.0007658	total: 1.92s	remaining: 357ms
843:	learn: 465.9527509	total: 1.92s	remaining: 355ms
844:	learn: 465.9311546	total: 1.92s	remaining: 352ms
845:	learn: 465.8478008	total: 1.92s	remaining: 350ms
846:	learn: 465.7050488	total: 1.93s	remaining: 348ms
847:	learn: 465.6728414	total: 1.93s	remaining: 346ms
848:	learn: 465.5647929	total: 1.93s	remaining: 343ms
849:	learn: 465.5281709	total: 1.93s	remaining: 341ms
850:	learn: 465.4460965	total: 1.94s	remaining: 339ms
851:	learn: 465.3668388	total: 1.94s	remaining: 337ms
852:	learn: 465.2521544	total: 1.94s	remaining: 334ms
853:	learn: 465.1434682	total: 1.94s	remaining: 333ms
854:	learn: 465.1097874	total: 1.95s	remaining: 330ms
855:	learn: 465.0188774	total: 1.95s	remaining: 328ms
856:	learn: 464.9593636	total: 1.95s	remaining: 326ms
857:	learn: 464.8560545	total: 1.95s	remaining: 324ms
858:	learn: 464.7866999	tota

In [35]:
# Model predictions

predictions = model.predict(X_test)

In [36]:
# RMSE

real_error = round(mean_squared_error(y_test, predictions)**0.5, 5)
real_error

531.90491

-------------------------------------------------------------------------------------------------------------------

**Test predictions**

In [37]:
diamonds_test

,carat,good_color,uniform,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,clarity_I1,clarity_IF,...,clarity_VS2,clarity_VVS1,clarity_VVS2,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,0.79,2,0.623090,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1.20,1,0.606676,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1.57,2,0.624317,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.90,2,0.636215,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.50,2,0.626719,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,2,0.620301,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
13481,0.71,2,0.621291,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
13482,0.70,2,0.618214,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
13483,0.70,2,0.585739,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [38]:
# Diamonds to be predicted

X = diamonds_test

In [39]:
#Scaling

scaler = RobustScaler()
X = scaler.fit_transform(X)
X

array([[ 0.140625  ,  0.        ,  0.33724739, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78125   , -1.        , -0.71893347, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.359375  ,  0.        ,  0.4161993 , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.02346698, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -2.066234  , ...,  0.        ,
         0.        ,  0.        ],
       [-0.46875   ,  0.        ,  0.23908483, ...,  0.        ,
         1.        ,  0.        ]])

In [40]:
# Predictions

predictions = model.predict(X)

In [41]:
# Formatting for Kaggle

predictions = pd.DataFrame(predictions, columns = ['Price'])
predictions.index.names = ['id']
predictions

,Price
id,
0,2796.279569
1,5853.593349
2,9770.308194
3,3928.361404
4,1698.431946
...,...
13480,1689.044134
13481,2332.079703
13482,3135.643338


In [42]:
# Creation of file where predictions are saved

predictions.to_csv('./results_catb.csv')